In [1]:
import RedCap
from importlib import reload


# This will reload the RedCap module when if there is change 
reload(RedCap)

<module 'RedCap' from 'C:\\Users\\sinel\\Documents\\PythonProjects\\REDCap Backup\\RedCap.py'>

In [2]:
import json
my_tokens = None

# Open the Json file for reading which contain API tokens 
with open('api_keys.json', mode = 'r') as file:
    # Load the JSON data from file
    my_tokens = json.load(file)
len(my_tokens.items())

16

> ### <font color=#42f5c2>Summary count</font>


In [3]:
# Summary count of
print(f'Project_Catagory {" "*18} Count \n{"_"*40}')
for key in my_tokens.keys():
    print(f'{key} : {" "*(35-len(key))} {len(my_tokens[key])}')


Project_Catagory                    Count 
________________________________________
ACHIDES_RDT_Comparison :               18
ACHIDES_Deletion_Surveillance_1 :      7
ACHIDES_Deletion_Surveillance_2 :      11
ACHIDES_Deletion_Surveillance_3 :      13
ACHIDES_Deletion_Surveillance_4 :      5
VISPA :                                3
PvSTATEM :                             3
TES2023 :                              3
TES2021 :                              5
tMDA_RCD :                             6
EMAGEN :                               7
Bioinformatics :                       2
INDIE :                                10
Urban_Malaria_1 :                      1
Urban_Malaria_2 :                      5
REDCap_8_11_5 :                        1


> ### <font color=#42f5c2>Call projectName function to get RECap projects name using token</font>


In [4]:
# Import user defined module called 'REDCap'
'''
There is a function called "project_name()" 
It will take the API keys and loads the project name assigned to it
'''
from REDCap import project_name
projects  = project_name(my_tokens)
print(projects.keys())

dict_keys(['ACHIDES_RDT_Comparison', 'ACHIDES_Deletion_Surveillance_1', 'ACHIDES_Deletion_Surveillance_2', 'ACHIDES_Deletion_Surveillance_3', 'ACHIDES_Deletion_Surveillance_4', 'VISPA', 'PvSTATEM', 'TES2023', 'TES2021', 'tMDA_RCD', 'EMAGEN', 'Bioinformatics', 'INDIE', 'Urban_Malaria_1', 'Urban_Malaria_2', 'REDCap_8_11_5'])


> ### <font color=#42f5c2>The  code below  takes user input of export format</font>
> <ul>
    <li>1 - CSV Data Raw</li>
    <li>2 - CSV Data Label</li>
    <li>3 - CSV Data Dictionary</li>
    <li>4 - XML Metadata Only</li>
    <li>5 - XML Metadata and Data</li>
    <li>6 - All</li>
</ul>


In [1]:
import Exceptions
input_message = ''' Choose the Format of Data:
                                           1 - CSV Data Raw
                                           2 - CSV Data Label
                                           3 - CSV Data Dictionary
                                           4 - XML Metadata Only
                                           5 - XML Metadata and Data
                                           6 - All'''
format_list = {1:'csv raw', 2:'csv label', 3:'csv data dictionary', 4:'xml meta data', 5:'xml metadata and data', 6:'all'}





export_format = None

while True:
    try:
        export_format = input(input_message)
        if not export_format.isdigit():
            input_message = ''
        export_format = int(export_format)
        if export_format < 1 or export_format > 6:
            input_message = ''
            raise Exceptions.InputError()
    except ValueError:
        print('Error: Enter a valid selection')
    except Exceptions.InputError as e:
        print(f'Error: {e}')
    else: 
        break

print(f'{format_list[export_format]} Selected')
export_format = format_list[export_format]
export_format
        

 Choose the Format of Data:
                                           1 - CSV Data Raw
                                           2 - CSV Data Label
                                           3 - CSV Data Dictionary
                                           4 - XML Metadata Only
                                           5 - XML Metadata and Data
                                           6 - All7
Error: Input must be 1 upto 6
1
csv raw Selected


'csv raw'

> ### <font color=#42f5c2>Creating Necessary Directories in Local Computer</font>


In [7]:
import os
import requests
import csv
import re
from datetime import date


url = 'https://redcapsvr2.ahri.gov.et/api/'

current_date = date.today().strftime('%B-%d-%Y')
root = 'E:\\MNTD\Backup\\'+current_date
sub_root_1 = ''
sub_root_2 = ''
sub_root_3 = ''
file_paths = dict()

if not os.path.exists(root):
    os.makedirs(root)
os.chdir(root)

for folder in projects.keys():
    sub_root_1 = folder
    if not os.path.exists(sub_root_1):
        os.makedirs(sub_root_1)
    for project, token in zip(projects[folder], my_tokens[folder]): 
        if re.findall(r'[\/:?"<>|]', project):
              project = re.sub(r'[\\/:?"<>|]','_',project)
        # print(project)
        sub_root_2 = sub_root_1 + '\\' + project.strip()
               
        if not os.path.exists(sub_root_2):   #create directory in local computer if it doesn't exist
            os.makedirs(sub_root_2)
        file_paths[token] = sub_root_2    


print('done')

done


In [14]:
switch_format = {
            'csv data raw' : csv_data_raw,
            'csv data label' : csv_data_label,
            'csv data dictionary': csv_data_dictionary,
            'xml metadata only': xml_metadata_only,
            'xml metadata and data' : xml_metadata_and_data,
            'all': all_exporter
}

selected_format = switch_format.get(export_format)
count = selected_format(file_paths)
print(f'Summary: {count} Total files saved successfully')

CSV Data Raw 101 files saved successfully
CSV Data Label 101 files saved successfully
CSV Data Dictionary 101 files saved successfully
XML Metadata Only 100 files saved successfully
XML Metadata and Data 100 files saved successfully
Summary : 503 Total files saved successfully


In [8]:
def csv_data_raw(file_paths):
    count = 0
    with requests.Session() as session:
        for token, file_path in file_paths.items():
            #The base name of a path is the last component of the path
            file_name = os.path.basename(file_path)
            file_path = os.path.join(file_path, 'CSV Data Raw')
            if not os.path.exists(file_path):
                os.makedirs(file_path)

            file_path = os.path.join(file_path, file_name)[:215] + '.csv'

            data = {
                'token': token,
                'content': 'record',
                'action': 'export',
                'format': 'csv',
                'type': 'flat',
                'csvDelimiter': '',
                'rawOrLabel': 'raw',
                'rawOrLabelHeaders': 'raw',
                'exportCheckboxLabel': 'false',
                'exportSurveyFields': 'false',
                'exportDataAccessGroups': 'false', 
                'returnFormat': 'json'
            }
            csv_data = session.post(url,data=data)

            if csv_data.status_code == 200:   
                with open(file_path, mode='wb') as file:
                    file.write(csv_data.content)
                    count += 1
            else:
                print(f'CSV Data Raw : {count} files Saved! \n But Something Wrong with Loading {file_name}')
                
    print(f'CSV Data Raw: {count} files saved successfully')
    return count

In [9]:
def csv_data_label(file_paths):
    count = 0
    with requests.Session() as session:
        for token, file_path in file_paths.items():
            #The base name of a path is the last component of the path
            file_name = os.path.basename(file_path)
            file_path = os.path.join(file_path, 'CSV Data Label')
            if not os.path.exists(file_path):
                os.makedirs(file_path)

            file_path = os.path.join(file_path, file_name)[:215] + '.csv'

            data = {
                'token': token,
                'content': 'record',
                'action': 'export',
                'format': 'csv',
                'type': 'flat',
                'csvDelimiter': '',
                'rawOrLabel': 'label',
                'rawOrLabelHeaders': 'raw',
                'exportCheckboxLabel': 'false',
                'exportSurveyFields': 'false',
                'exportDataAccessGroups': 'false', 
                'returnFormat': 'json'
            }
            csv_data = session.post(url,data=data)

            if csv_data.status_code == 200:   
                with open(file_path, mode='wb') as file:
                    file.write(csv_data.content)
                    count += 1
            else:
                print(f'CSV Data Label: {count} files Saved! \n But Something Wrong with Loading {file_name}')
                
    print(f'CSV Data Label: {count} files saved successfully')
    return count

In [10]:
def csv_data_dictionary(file_paths):
    count = 0
    with requests.Session() as session:
        for token, file_path in file_paths.items():
            #The base name of a path is the last component of the path
            file_name = os.path.basename(file_path)
            file_path = os.path.join(file_path, 'CSV Data Dictionary')
            if not os.path.exists(file_path):
                os.makedirs(file_path)

            file_path = os.path.join(file_path, file_name)[:215] + '.csv'

            data = {
                'token': token,
                'content': 'metadata',
                'format': 'csv',
                'returnFormat': 'csv'
            }
            csv_data = session.post(url,data=data)

            if csv_data.status_code == 200:   
                with open(file_path, mode='wb') as file:
                    file.write(csv_data.content)
                    count += 1
            else:
                print(f'CSV Data Dictionary: {count} files Saved! \n But Something Wrong with Loading {file_name}')
                
    print(f'CSV Data Dictionary: {count} files saved successfully')
    return count

In [11]:
def xml_metadata_only(file_path):
    count = 0
    with requests.Session() as session:
        for token, file_path in file_paths.items():
            #The base name of a path is the last component of the path
            file_name = os.path.basename(file_path)
            file_path = os.path.join(file_path, 'XML Metadata Only')
            if not os.path.exists(file_path):
                os.makedirs(file_path)

            file_path = os.path.join(file_path, file_name)[:215] + '.xml'

            data_config = {'token' : token,
                          'content' : 'project_xml',
                          'format' : 'xml',
                          'returnMetadataOnly' : 'true',
                          'exportFiles' : 'false',
                          'exportSurveyFields' : 'false',
                          'exportDataAccessGroups' : 'false',
                          'returnFormat' : 'json'
                            }
            xml_data = session.post(url, data = data_config)

            # write xml to local storage
            if xml_data.status_code == 200:
                with open(file_path, mode = 'wb') as file:
                    file.write(xml_data.content)
                    count += 1
            else:
                print(f'XML Metadata Only: {count} files Saved! \n But Something Wrong with Loading {file_name}')
                
    print(f'XML Metadata Only: {count} files saved successfully')
    return count
     

In [12]:
def xml_metadata_and_data(file_path):
    count = 0
    with requests.Session() as session:
        for token, file_path in file_paths.items():
            #The base name of a path is the last component of the path
            file_name = os.path.basename(file_path)
            file_path = os.path.join(file_path, 'XML Metadata and Data')
            if not os.path.exists(file_path):
                os.makedirs(file_path)

            file_path = os.path.join(file_path, file_name)[:215] + '.xml'

            data_config = {'token' : token,
              'content' : 'project_xml',
              'format' : 'xml',
              'returnMetadataOnly' : 'false',
              'exportFiles' : 'false',
              'exportSurveyFields' : 'false',
              'exportDataAccessGroups' : 'false',
              'returnFormat' : 'json'
                }
            xml_data = session.post(url, data = data_config)

            # write xml to local storage
            if xml_data.status_code == 200:
                with open(file_path, mode = 'wb') as file:
                    file.write(xml_data.content)
                    count += 1
            else:
                print(f'XML Metadata and Data: {count} files Saved! \n But Something Wrong with Loading {file_name}')
                
    print(f'XML Metadata and Data: {count} files saved successfully')
    return count
     

In [13]:
def all_exporter(file_path):
    count  = csv_data_raw(file_path) + csv_data_label(file_path) + csv_data_dictionary(file_path) + xml_metadata_only(file_path) + xml_metadata_and_data(file_path)
    return count